In [323]:
# Code to download data from https://ember-climate.org/app/uploads/2022/09/european_wholesale_electricity_price_data_daily-4.csv

import pandas as pd
import plotly_express as px

# Select data to chart
data_frequency = 3 # Enter 1 (daily prices), 2 (monthly average prices), 3 (yearly average prices)
country_coverage = 2 # Enter 1 (all countries), 2 (specific countries that need to be listed in 'country_list' variable below)
country_list = ['Germany', 'France', 'Italy'] # for use when user wants to import data for certain countries (i.e. option 2 in 'country_coverage')


def time_series_data_func(df, country_list): # returns time series data (for charting) as well as geographical data (for mapping) of daily/monthly/yearly prices either for all countries or for some. 
    # save ISO3 country codes in a DataFrame. ISO3 codes will be used later to create geographical map chart
    country_names  = list(df['Country'].unique()) # assign unique country names to country_names list
    country_codes = list(df['ISO3 Code'].unique()) # assign unique ISO3 country codes to country_codes list
    df_ISO_codes = pd.DataFrame({'Country': country_names, 'ISO3 Code': country_codes}) # create 'df_ISO_codes' DataFrame to host country names and their ISO3 codes
    if country_coverage == 2: # filters time series data only for certain countries that were selected in 'country_list' variable
        df = df[df['Country'].isin(country_list)]
    if data_frequency == 1: # returns time series data (for charting) as well as geographical data (for mapping) of all daily prices either for all countries or for some. 
        df_geo = geo_data_func(df, df_ISO_codes) # formats data that will be used later to create geographical map chart
        return df, df_geo
    elif data_frequency == 2 or data_frequency == 3: # returns time series data (for charting) as well as geographical data (for mapping) of average monthly/yearly prices either for all countries or for some. 
        # calculate monthly average prices by country
        df = df.pivot(index='Date', columns='Country', values='Price (EUR/MWhe)') # move country names to columns
        if data_frequency == 2:
            df = df.groupby(pd.PeriodIndex(df.index, freq="M")).mean() # calculate monthly average values per country
        elif data_frequency == 3:
            df = df.groupby(pd.PeriodIndex(df.index, freq="Y")).mean() # calculate monthly average values per country
        df = df.reset_index() # move 'Date' column from being index into a separate column
        df = df.melt(id_vars=['Date'], value_vars=list(df.columns)[1:], var_name='Country', value_name='Price (EUR/MWhe)') # move individual country columns into a single 'Country' column
        df['Date'] = df['Date'].dt.to_timestamp() # convert DataFrame periods (e.g. 2015-01) to timestamp (e.g. 2015-01-01)
        df_geo = geo_data_func(df, df_ISO_codes) # formats data that will be used later to create geographical map chart
        return df, df_geo

def geo_data_func(df, df_ISO_codes): # formats data that will be used later to create geographical map chart
    if data_frequency == 1:
        df_geo = df
    elif data_frequency == 2 or data_frequency == 3:
        df_geo = pd.merge(df, df_ISO_codes, on ='Country', how ='left').dropna() # add 'ISO3 Code' column to 'df_avg_m' DataFrame. This will be used later to create a geographic map chart. Also, uses 'dropna()' function to remove NaN values to prevent geographic chart from showing wrong colours due to NaN values.
    df_geo = df_geo.sort_values('Date').groupby('Country').tail(1) # filter to show the latest (by 'Date' column) value per country
    df_geo = df_geo.sort_values('Price (EUR/MWhe)') # sort by values
    return df_geo


def line_chart_func(df): # creates chart of the latest daily or latest average monthly/yearly prices by country
    fig = px.line(df, 
        x=df['Date'],
        y=df['Price (EUR/MWhe)'],
        color=df['Country'],
        height=500, 
        width=900,
        template='plotly_dark'
        )  # charts the data
    fig.show() # chart daily the data

# create geographic map of the latest daily or latest average monthly/yearly prices by country
def geo_map_func(df):
    map_fig = px.scatter_geo(df,
                            locations='ISO3 Code',
                            projection='orthographic',
                            color='Price (EUR/MWhe)',
                            height=500, 
                            width=900,
                            color_continuous_scale=['green', 'yellow', 'red'],
                            #color_continuous_scale=px.colors.sequential.Viridis,
                            #color_continuous_scale='Bluered_r',
                            opacity=0.8,
                            hover_name='Country',
                            hover_data=['Country', 'Price (EUR/MWhe)']
    )
    map_fig.show()



# download data from CSV into DataFrame
df = pd.read_csv('european_wholesale_electricity_price_data_daily-4.csv') # copy CSV file downloaded from https://ember-climate.org/app/uploads/2022/09/european_wholesale_electricity_price_data_daily-4.csv into 'df' DataFrame into the same directory as this python file

# functions to run

time_series_data = time_series_data_func(df, country_list)
time_series_chart = line_chart_func(time_series_data[0])
geo_map= geo_map_func(time_series_data[1])

In [315]:
import pandas as pd
import plotly_express as px

# download data from CSV into DataFrame
df = pd.read_csv('european_wholesale_electricity_price_data_daily-4.csv') # copy CSV file downloaded from https://ember-climate.org/app/uploads/2022/09/european_wholesale_electricity_price_data_daily-4.csv into 'df' DataFrame into the same directory as this python file
df

,Country,ISO3 Code,Date,Price (EUR/MWhe)
0,Austria,AUT,2015-01-01,35.86
1,Austria,AUT,2015-01-02,35.86
2,Austria,AUT,2015-01-03,35.86
3,Austria,AUT,2015-01-04,35.86
4,Austria,AUT,2015-01-05,36.18
...,...,...,...,...
78387,Sweden,SWE,2022-11-26,218.72
78388,Sweden,SWE,2022-11-27,133.71
78389,Sweden,SWE,2022-11-28,191.90
78390,Sweden,SWE,2022-11-29,357.70


In [211]:
# calculate monthly average prices by country
df_avg_a = df.pivot(index='Date', columns='Country', values='Price (EUR/MWhe)') # move country names to columns
df_avg_a = df_avg_a.groupby(pd.PeriodIndex(df_avg_a.index, freq="Y")).mean() # calculate monthly average values per country
df_avg_a = df_avg_a.reset_index() # move 'Date' column from being index into a separate column
df_avg_a = df_avg_a.melt(id_vars=['Date'], value_vars=list(df_avg_a.columns)[1:], var_name='Country', value_name='Price (EUR/MWhe)') # move individual country columns into a single 'Country' column
df_avg_a['Date'] = df_avg_a['Date'].dt.to_timestamp() # convert DataFrame periods (e.g. 2015-01) to timestamp (e.g. 2015-01-01)
#df_avg_m

# chart monthly average prices by country
fig = px.line(df_avg_a, 
    x=df_avg_a['Date'],
    y=df_avg_a['Price (EUR/MWhe)'],
    color=df_avg_a['Country'],
    height=500, 
    width=900,
    template='plotly_dark'
    )  # charts the data
fig.show() # chart daily the data

In [205]:
# calculate monthly average prices by country
df_avg_m = df.pivot(index='Date', columns='Country', values='Price (EUR/MWhe)') # move country names to columns
df_avg_m = df_avg_m.groupby(pd.PeriodIndex(df_avg_m.index, freq="M")).mean() # calculate monthly average values per country
df_avg_m = df_avg_m.reset_index() # move 'Date' column from being index into a separate column
df_avg_m = df_avg_m.melt(id_vars=['Date'], value_vars=list(df_avg_m.columns)[1:], var_name='Country', value_name='Price (EUR/MWhe)') # move individual country columns into a single 'Country' column
df_avg_m['Date'] = df_avg_m['Date'].dt.to_timestamp() # convert DataFrame periods (e.g. 2015-01) to timestamp (e.g. 2015-01-01)
#df_avg_m

# chart monthly average prices by country
fig = px.line(df_avg_m, 
    x=df_avg_m['Date'],
    y=df_avg_m['Price (EUR/MWhe)'],
    color=df_avg_m['Country'],
    height=500, 
    width=900,
    template='plotly_dark'
    )  # charts the data
fig.show() # chart daily the data

In [208]:
# create a map of latest average monthly prices by country

# save ISO3 country codes in a DataFrame. ISO3 codes will be used later to create geographical map chart
country_names  = list(df['Country'].unique()) # assign unique country names to country_names list
country_codes = list(df['ISO3 Code'].unique()) # assign unique ISO3 country codes to country_codes list
df_ISO_codes = pd.DataFrame({'Country': country_names, 'ISO3 Code': country_codes}) # create 'df_ISO_codes' DataFrame to host country names and their ISO3 codes
df_geo_avg = pd.merge(df_avg_m, df_ISO_codes, on ='Country', how ='left').dropna() # add 'ISO3 Code' column to 'df_avg_m' DataFrame. This will be used later to create a geographic map chart. Also, uses 'dropna()' function to remove NaN values to prevent geographic chart from showing wrong colours due to NaN values.

df_geo_avg = df_geo_avg.sort_values('Date').groupby('Country').tail(1) # filter to show the latest (by 'Date' column) value per country
df_geo_avg = df_geo_avg.sort_values('Price (EUR/MWhe)') # sort by values

# create geographic map chart
map_fig = px.scatter_geo(df_geo_avg,
                        locations='ISO3 Code',
                        projection='orthographic',
                        color='Price (EUR/MWhe)',
                        height=500, 
                        width=900,
                        color_continuous_scale=['green', 'yellow', 'red'],
                        #color_continuous_scale=px.colors.sequential.Viridis,
                        #color_continuous_scale='Bluered_r',
                        opacity=0.8,
                        hover_name='Country',
                        hover_data=['Country', 'Price (EUR/MWhe)']
)
map_fig.show()
